In [18]:
import pandas as pd
import pickle
from pathlib import Path

import json

In [9]:
data_RAW = pd.read_csv("assets/data/heart.csv")

features = data_RAW.drop(columns="target")
target = data_RAW["target"]

In [11]:
models = {}

for item in Path("classifiers").iterdir():
    name = item.stem.replace("-", "_")
    estimator = pickle.load(item.open("rb"))
    models[name] = estimator

In [3]:
models.keys()

dict_keys(['decision_tree', 'knn', 'random_forest', 'svm'])

In [81]:
def get_prediction(features, prob=False):

    if features.shape != (1, 13):
        raise ValueError(f"Expecting a single row with 13 features. Got {features.shape} instead of (1, 13)")

    result = {}

    for name, estimator in models.items():
        if prob:
            prediction = (estimator.predict_proba(features) * 100).astype(int)
        else:
            prediction = estimator.predict(features).item()

        result[name] = prediction

    return result

In [82]:
def get_predictions(features, prob=False):

    if features.ndim != 2 or features.shape[1] != 13 or features.shape[0] == 1:
        raise ValueError(f"Expecting atleast two rows with 13 features. Got {features.shape} instead of (* > 1, 13)")

    results = {}

    for idx, row in features.iterrows():
        row = row.to_frame().T
        results[idx] = get_prediction(row, prob=prob)

    return results

In [91]:
data = json.loads(
    """{
    "age": 100,
    "sex": 1,
    "cp": 0,
    "trestbps": 115,
    "chol": 258,
    "fbs": 0,
    "restecg": 0,
    "thalach": 141,
    "exang": 1,
    "oldpeak": 2.8,
    "slope": 0,
    "ca": 1,
    "thal": 2
}"""
)

data

{'age': 100,
 'sex': 1,
 'cp': 0,
 'trestbps': 115,
 'chol': 258,
 'fbs': 0,
 'restecg': 0,
 'thalach': 141,
 'exang': 1,
 'oldpeak': 2.8,
 'slope': 0,
 'ca': 1,
 'thal': 2}

In [92]:
data = pd.DataFrame([data])
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,100,1,0,115,258,0,0,141,1,2.8,0,1,2


In [93]:
get_prediction(data, True)

{'decision_tree': array([[63, 36]]),
 'knn': array([[88, 11]]),
 'random_forest': array([[62, 37]]),
 'svm': array([[64, 35]])}

In [94]:
data = json.loads(
    """[
    {
    "age": 100,
    "sex": 1,
    "cp": 0,
    "trestbps": 115,
    "chol": 258,
    "fbs": 0,
    "restecg": 0,
    "thalach": 141,
    "exang": 1,
    "oldpeak": 2.8,
    "slope": 0,
    "ca": 1,
    "thal": 2
},
{
        "age": 63,
        "sex": 1,
        "cp": 3,
        "trestbps": 145,
        "chol": 233,
        "fbs": 1,
        "restecg": 0,
        "thalach": 150,
        "exang": 0,
        "oldpeak": 2.3,
        "slope": 0,
        "ca": 0,
        "thal": 1
        }
    ]
    
"""
)

data

[{'age': 100,
  'sex': 1,
  'cp': 0,
  'trestbps': 115,
  'chol': 258,
  'fbs': 0,
  'restecg': 0,
  'thalach': 141,
  'exang': 1,
  'oldpeak': 2.8,
  'slope': 0,
  'ca': 1,
  'thal': 2},
 {'age': 63,
  'sex': 1,
  'cp': 3,
  'trestbps': 145,
  'chol': 233,
  'fbs': 1,
  'restecg': 0,
  'thalach': 150,
  'exang': 0,
  'oldpeak': 2.3,
  'slope': 0,
  'ca': 0,
  'thal': 1}]

In [95]:
data = pd.DataFrame(data)

In [98]:
get_predictions(data, False)

{0: {'decision_tree': 0, 'knn': 0, 'random_forest': 0, 'svm': 1},
 1: {'decision_tree': 1, 'knn': 0, 'random_forest': 1, 'svm': 1}}